In [ ]:
%load_ext autoreload
%autoreload 2

Some imports:

In [ ]:
import numpy as np
import scipy as sc
from numpy.random import rand
from time import time
import swarming

from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import row
output_notebook()

Main model object:

In [ ]:
swarm = swarming.InitialCondition(
    condition="randomspeed",
    n_particles=500,
    x_range=(-30, 30),
    y_range=(-30, 30),
    rhs=swarming.rep_attr_rhs
)

Scale the length of velocity-arrows:

In [ ]:
swarm.VSCALE=2

Some initial conditions:

In [ ]:
show(swarm.plot_initial(ncols=4))

Show current state of model:

In [ ]:
nh = show(
    row(swarm.plot(plot_mean=False), swarm.plot_density(size=5)),
    notebook_handle=True
)

Evolve a few steps:

In [ ]:
swarm.cds.data = swarm.evolve(0.1, 10, snapshot=False).cds_dict()
push_notebook(handle=nh)
swarm.time

Change some parameters:

In [ ]:
params = {"lr": .2, "la": 100, "Ca": 20, "Cr":23}
# params = {"const": 0.01, "damping":0.5}
swarm.cds.data = swarm.evolve(0.1, 100, snapshot=False, **params).cds_dict()
push_notebook(handle=nh)
swarm.time

Record for a given time, snapshotting every ``n_steps`` until ``max_time`` is reached making ``time_step``s:

In [ ]:
swarm.record_for_time(100.0, 0.25, 4)

record for 200 history points:

In [ ]:
swarm.record_until(100, 0.25, 4)

Final state density plot:

In [ ]:
show(swarm.plot_density(size=4))

all recorded states:

In [ ]:
swarm.history

Showing final state, trajectory of center of gravity, development of changerates:

In [ ]:
dimensions = {"plot_width": 300, "plot_height": 300}

chgx, chgv, tim = swarm.calc_chg_from_history()

f1 = figure(title="Changing mean x over time", **dimensions)
f1.line(x=tim, y=chgx)
f2 = figure(title="Changing mean v over time", **dimensions)
f2.line(x=tim, y=chgv)

show(row(
    swarm.plot(plot_mean=True),
    swarm.plot_trajectory(),
    f1,
    f2
))

Reset state:

In [ ]:
swarm.set_initial("square")